In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import time
import csv

In [2]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
output = loss(input, target)
output.backward()

tensor([[ 0.7737, -0.2971, -0.3081,  0.2080, -0.3426],
        [ 0.9303, -0.1958,  0.4231, -0.1862, -0.6831],
        [ 0.7970, -0.5578,  0.0606,  0.1183,  0.4745]], requires_grad=True)
tensor([2, 3, 3])
tensor([[-1.9185,  0.7913, -1.3450, -0.1389,  0.5277],
        [ 0.2598, -0.8366, -0.6156, -1.1807, -0.4429],
        [ 0.0677, -0.1193, -0.2395, -1.3903,  1.8835]], requires_grad=True)
tensor([[0.5405, 0.1716, 0.1690, 0.1094, 0.0095],
        [0.0694, 0.4841, 0.2661, 0.0684, 0.1120],
        [0.1910, 0.1282, 0.4392, 0.0765, 0.1652]])


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


In [9]:
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        # print("pred: ",pred.shape)
        # print("y:",y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    epoch_start = time.perf_counter()
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295818  [   64/60000]
loss: 2.290596  [ 6464/60000]
loss: 2.271947  [12864/60000]
loss: 2.264305  [19264/60000]
loss: 2.256371  [25664/60000]
loss: 2.214496  [32064/60000]
loss: 2.216768  [38464/60000]
loss: 2.187126  [44864/60000]
loss: 2.177625  [51264/60000]
loss: 2.139527  [57664/60000]
Test Error: 
 Accuracy: 54.3%, Avg loss: 2.144349 

Epoch 2
-------------------------------
loss: 2.147761  [   64/60000]
loss: 2.150087  [ 6464/60000]
loss: 2.088532  [12864/60000]
loss: 2.104580  [19264/60000]
loss: 2.069278  [25664/60000]
loss: 1.991538  [32064/60000]
loss: 2.017007  [38464/60000]
loss: 1.942643  [44864/60000]
loss: 1.942232  [51264/60000]
loss: 1.865337  [57664/60000]
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.870834 

Epoch 3
-------------------------------
loss: 1.896447  [   64/60000]
loss: 1.877945  [ 6464/60000]
loss: 1.756815  [12864/60000]
loss: 1.799840  [19264/60000]
loss: 1.701895  [25664/60000]
loss: 1.638587  [32064/600

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>